#### This is more of a dev notebook for testing and debugging but some may find it useful, so it is provided here for now.

In [4]:
import MDAnalysis as mda
import numpy as np

from MDAnalysis.units import constants
from MDAnalysis.transformations import set_dimensions

In [5]:
# prev code, not used directly, only here for reference

# Step trajectory of unit velocities i.e. v = 0 at t = 0,
# v = 1 at t = 1, v = 2 at t = 2, etc. for all components x, y, z
def step_vtraj(NSTEP):
    v = np.arange(NSTEP)
    velocities = np.vstack([v, v, v]).T
    # NSTEP frames x 1 atom x 3 dimensions, where NSTEP = 5001
    velocities_reshape = velocities.reshape([NSTEP, 1, 3])
    u = mda.Universe.empty(1, n_frames=NSTEP, velocities=True)
    for i, ts in enumerate(u.trajectory):
        u.atoms.velocities = velocities_reshape[i]
    return u

# Position trajectory corresponding to unit velocity trajectory
def step_vtraj_pos(NSTEP):
    x = np.arange(NSTEP).astype(np.float64)
    # Since initial position and velocity are 0 and acceleration is 1,
    # position = 1/2t^2
    x *= x / 2
    positions = np.vstack([x, x, x]).T
    # NSTEP frames x 1 atom x 3 dimensions, where NSTEP = 5001
    positions_reshape = positions.reshape([NSTEP, 1, 3])
    u_pos = mda.Universe.empty(1)
    u_pos.load_new(positions_reshape)
    return u_pos

In [6]:
# debugging version of Helfand viscosity class with some extra print statements
from MDAnalysis.analysis.base import AnalysisBase
from MDAnalysis.core.groups import UpdatingAtomGroup
from MDAnalysis.exceptions import NoDataError

class ViscosityHelfand(AnalysisBase):
    """
    Class to calculate viscosity using the Einstein-Helfand approach.

    Parameters
    ----------
    atomgroup : AtomGroup
        An MDAnalysis :class:`~MDAnalysis.core.groups.AtomGroup`.
        Note that :class:`UpdatingAtomGroup` instances are not accepted.
    temp_avg : float (optional, default 300)
        Average temperature over the course of the simulation, in Kelvin.
    dim_type : {'xyz', 'xy', 'yz', 'xz', 'x', 'y', 'z'}
        Desired dimensions to be included in the viscosity calculation.
        Defaults to 'xyz'.

    Attributes
    ----------
    atomgroup : :class:`~MDAnalysis.core.groups.AtomGroup`
        The atoms to which this analysis is applied
    dim_fac : int
        Dimensionality :math:`d` of the viscosity computation.
    results.timeseries : :class:`numpy.ndarray`
        The averaged viscosity over all the particles with respect to lag-time.
        Obtained after calling :meth:`ViscosityHelfand.run`
    results.visc_by_particle : :class:`numpy.ndarray`
        The viscosity of each individual particle with respect to lag-time.
    start : Optional[int]
        The first frame of the trajectory used to compute the analysis.
    stop : Optional[int]
        The frame to stop at for the analysis.
    step : Optional[int]
        Number of frames to skip between each analyzed frame.
    n_frames : int
        Number of frames analysed in the trajectory.
    n_particles : int
        Number of particles viscosity was calculated over.
    """

    def __init__(
        self,
        atomgroup: "AtomGroup",
        temp_avg: float = 300.0,
        dim_type: str = "xyz",
        **kwargs,
    ) -> None:
        # the below line must be kept to initialize the AnalysisBase class!
        super().__init__(atomgroup.universe.trajectory, **kwargs)
        # after this you will be able to access `self.results`
        # `self.results` is a dictionary-like object
        # that can should used to store and retrieve results
        # See more at the MDAnalysis documentation:
        # https://docs.mdanalysis.org/stable/documentation_pages/analysis/base.html?highlight=results#MDAnalysis.analysis.base.Results

        if isinstance(atomgroup, UpdatingAtomGroup):
            raise TypeError(
                "UpdatingAtomGroups are not valid for viscosity computation"
            )

        # args
        self.temp_avg = temp_avg
        self.dim_type = dim_type.lower()
        self._dim, self.dim_fac = self._parse_dim_type(self.dim_type)
        # self.fft = fft # consider whether fft is possible later

        # local
        self.atomgroup = atomgroup
        self.n_particles = len(self.atomgroup)

    def _prepare(self):
        """
        Set up viscosity, mass, velocity, and position arrays
        before the analysis loop begins
        """
        # 2D viscosity array of frames x particles
        self.results.visc_by_particle = np.zeros(
            (self.n_frames, self.n_particles)
        )

        self._volumes = np.zeros((self.n_frames))

        self._masses = self.atomgroup.masses
        self._masses_rs = self._masses.reshape((1, len(self._masses), 1))

        # 3D arrays of frames x particles x dimensionality
        # for velocities and positions
        self._velocities = np.zeros(
            (self.n_frames, self.n_particles, self.dim_fac)
        )

        self._positions = np.zeros(
            (self.n_frames, self.n_particles, self.dim_fac)
        )
        # self.results.timeseries not set here

    @staticmethod
    def _parse_dim_type(dim_str):
        r"""Sets up the desired dimensionality of the calculation."""
        keys = {
            "x": [0],
            "y": [1],
            "z": [2],
            "xy": [0, 1],
            "xz": [0, 2],
            "yz": [1, 2],
            "xyz": [0, 1, 2],
        }

        try:
            _dim = keys[dim_str]
        except KeyError:
            raise ValueError(
                "invalid dim_type: {} specified, please specify one of xyz, "
                "xy, xz, yz, x, y, z".format(dim_str)
            )

        return _dim, len(_dim)

    def _single_frame(self):
        """
        Constructs arrays of velocities and positions
        for viscosity computation.
        """
        # This runs once for each frame of the trajectory

        # The trajectory positions update automatically
        # You can access the frame number using self._frame_index

        # trajectory must have velocity and position information
        if not (
            self._ts.has_velocities
            and self._ts.has_positions
            and self._ts.volume != 0
        ):
            raise NoDataError(
                "Helfand viscosity computation requires "
                "velocities, positions, and box volume in the trajectory"
            )

        # fill volume array
        self._volumes[self._frame_index] = self._ts.volume

        # set shape of velocity array
        self._velocities[self._frame_index] = self.atomgroup.velocities[
            :, self._dim
        ]

        # set shape of position array
        self._positions[self._frame_index] = self.atomgroup.positions[
            :, self._dim
        ]

    def _conclude(self):
        r"""Calculates viscosity via the simple "windowed" algorithm."""
        self._vol_avg = np.average(self._volumes)

        lagtimes = np.arange(1, self.n_frames)

        # iterate through all possible lagtimes from 1 to number of frames
        for lag in lagtimes:
            # get difference of momentum * position shifted by "lag" frames
            diff = (
                self._masses_rs
                * self._velocities[:-lag, :, :]
                * self._positions[:-lag, :, :]
                - self._masses_rs
                * self._velocities[lag:, :, :]
                * self._positions[lag:, :, :]
            )
            print(f"Diff: {diff}")

            # square and sum each x(, y, z) diff per particle
            sq_diff = np.square(diff).sum(axis=-1)

            print(f"Sq_diff: {sq_diff}")

            # average over # frames
            # update viscosity by particle array
            self.results.visc_by_particle[lag, :] = np.mean(sq_diff, axis=0)

        # divide by 2, boltzman constant, vol_avg, and temp_avg
        self.results.visc_by_particle = self.results.visc_by_particle / (
            2
            * constants["Boltzmann_constant"]
            * self._vol_avg
            * self.temp_avg
        )
        print(f"visc_by_particle: {self.results.visc_by_particle}")
        # average over # particles and update results array
        self.results.timeseries = self.results.visc_by_particle.mean(axis=1)

        print(self.results.timeseries.shape)
        print(self.results.timeseries.dtype)
        print(f"results.timeseries: {self.results.timeseries}")

In [7]:
# Step trajectory of unit velocities for a single particle i.e. v = 0 at t = 0,
# v = 1 at t = 1, v = 2 at t = 2, etc. for all components x, y, z
# contains additional info needed: masses, positions, box volume
def step_vtraj_full(NSTEP):
    # Set up unit velocities
    v = np.arange(NSTEP)
    velocities = np.vstack([v, v, v]).T
    # NSTEP frames x 1 atom x 3 dimensions, where NSTEP = 5001
    velocities_reshape = velocities.reshape([NSTEP, 1, 3])

    # Positions derived from unit velocity setup
    x = np.arange(NSTEP).astype(np.float64)
    # Since initial position and velocity are 0 and acceleration is 1,
    # position = 1/2t^2
    x *= x / 2
    positions = np.vstack([x, x, x]).T
    # NSTEP frames x 1 atom x 3 dimensions, where NSTEP = 5001
    positions_reshape = positions.reshape([NSTEP, 1, 3])
    u = mda.Universe.empty(1, n_frames=NSTEP, velocities=True)

    dim = [2, 2, 2, 90, 90, 90]
    for i, ts in enumerate(u.trajectory):
        u.atoms.velocities = velocities_reshape[i]
        u.atoms.positions = positions_reshape[i]
        set_dimensions(dim)(u.trajectory.ts)

    u.add_TopologyAttr('masses', [16.0])
    return u

# verify toy system works
test_u = step_vtraj_full(10)

print(test_u.atoms.masses)

vol_used = 0.0
for ts in test_u.trajectory:
    print(f"Positions: {test_u.atoms.positions}")
    print(f"Velocities: {test_u.atoms.velocities}")
    print(f"Volume: {ts.volume}")
    print(f"Positions: {ts.velocities}")
    vol_used = ts.volume

[16.]
Positions: [[0. 0. 0.]]
Velocities: [[0. 0. 0.]]
Volume: 8.0
Positions: [[0. 0. 0.]]
Positions: [[0.5 0.5 0.5]]
Velocities: [[1. 1. 1.]]
Volume: 8.0
Positions: [[1. 1. 1.]]
Positions: [[2. 2. 2.]]
Velocities: [[2. 2. 2.]]
Volume: 8.0
Positions: [[2. 2. 2.]]
Positions: [[4.5 4.5 4.5]]
Velocities: [[3. 3. 3.]]
Volume: 8.0
Positions: [[3. 3. 3.]]
Positions: [[8. 8. 8.]]
Velocities: [[4. 4. 4.]]
Volume: 8.0
Positions: [[4. 4. 4.]]
Positions: [[12.5 12.5 12.5]]
Velocities: [[5. 5. 5.]]
Volume: 8.0
Positions: [[5. 5. 5.]]
Positions: [[18. 18. 18.]]
Velocities: [[6. 6. 6.]]
Volume: 8.0
Positions: [[6. 6. 6.]]
Positions: [[24.5 24.5 24.5]]
Velocities: [[7. 7. 7.]]
Volume: 8.0
Positions: [[7. 7. 7.]]
Positions: [[32. 32. 32.]]
Velocities: [[8. 8. 8.]]
Volume: 8.0
Positions: [[8. 8. 8.]]
Positions: [[40.5 40.5 40.5]]
Velocities: [[9. 9. 9.]]
Volume: 8.0
Positions: [[9. 9. 9.]]


In [8]:
def characteristic_poly_helfand(total_frames, n_dim, temp_avg=300.0, vol_avg=8.0):
    result = np.zeros(total_frames)

    for lag in range(total_frames):
        sum = 0
        sum = np.dtype('float64').type(sum)

        print(lag)
        for curr in range((total_frames - lag)):
            # mass * velocities * positions
            print(f"Curr + lag: {1/2 * ((curr + lag) ** 2)}")
            print(f"Curr: {1/2 * (curr ** 2)}")
            helf_diff = (16.0 * (curr + lag) * 1/2 * ((curr + lag) ** 2)
                         - 16.0 * curr * 1/2 * (curr ** 2))
            sum += helf_diff ** 2
            print(f"Helf_diff {curr}: {helf_diff ** 2}")
        print(f"Sum lag {lag}: {sum}")
        vis_helf = sum * n_dim / (
            (total_frames - lag)
            * 2
            * constants["Boltzmann_constant"]
            * vol_avg
            * temp_avg
        )
        print(f"vis_helf lag {lag}: {vis_helf}")
        result[lag] = vis_helf
    print(result.shape)
    print(result.dtype)
    return result

test_res = characteristic_poly_helfand(10, 3)
print(test_res)

0
Curr + lag: 0.0
Curr: 0.0
Helf_diff 0: 0.0
Curr + lag: 0.5
Curr: 0.5
Helf_diff 1: 0.0
Curr + lag: 2.0
Curr: 2.0
Helf_diff 2: 0.0
Curr + lag: 4.5
Curr: 4.5
Helf_diff 3: 0.0
Curr + lag: 8.0
Curr: 8.0
Helf_diff 4: 0.0
Curr + lag: 12.5
Curr: 12.5
Helf_diff 5: 0.0
Curr + lag: 18.0
Curr: 18.0
Helf_diff 6: 0.0
Curr + lag: 24.5
Curr: 24.5
Helf_diff 7: 0.0
Curr + lag: 32.0
Curr: 32.0
Helf_diff 8: 0.0
Curr + lag: 40.5
Curr: 40.5
Helf_diff 9: 0.0
Sum lag 0: 0.0
vis_helf lag 0: 0.0
1
Curr + lag: 0.5
Curr: 0.0
Helf_diff 0: 64.0
Curr + lag: 2.0
Curr: 0.5
Helf_diff 1: 3136.0
Curr + lag: 4.5
Curr: 2.0
Helf_diff 2: 23104.0
Curr + lag: 8.0
Curr: 4.5
Helf_diff 3: 87616.0
Curr + lag: 12.5
Curr: 8.0
Helf_diff 4: 238144.0
Curr + lag: 18.0
Curr: 12.5
Helf_diff 5: 529984.0
Curr + lag: 24.5
Curr: 18.0
Helf_diff 6: 1032256.0
Curr + lag: 32.0
Curr: 24.5
Helf_diff 7: 1827904.0
Curr + lag: 40.5
Curr: 32.0
Helf_diff 8: 3013696.0
Sum lag 1: 6755904.0
vis_helf lag 1: 56426.98120793744
2
Curr + lag: 2.0
Curr: 0.0
He

In [9]:
# Expected viscosity function results for unit velocity trajectory
def characteristic_poly_helfand(total_frames, n_dim, temp_avg=300.0, vol_avg=8.0):
    result = np.zeros(total_frames)

    for lag in range(total_frames):
        sum = 0
        sum = np.dtype('float64').type(sum)

        print(f"lag: {lag}")
        for curr in range((total_frames - lag)):
            # mass * velocities * positions
            print(f"curr: {curr}")
            helf_diff = (16.0 * (curr + lag) * 1/2 * ((curr + lag) ** 2)
                         - 16.0 * curr * 1/2 * (curr ** 2))
            sum += helf_diff ** 2
        vis_helf = sum * n_dim / (
            (total_frames - lag)
            * 2
            * constants["Boltzmann_constant"]
            * vol_avg
            * temp_avg
        )
        result[lag] = vis_helf
    return result

test_res = characteristic_poly_helfand(10, 3)
print(test_res)

lag: 0
curr: 0
curr: 1
curr: 2
curr: 3
curr: 4
curr: 5
curr: 6
curr: 7
curr: 8
curr: 9
lag: 1
curr: 0
curr: 1
curr: 2
curr: 3
curr: 4
curr: 5
curr: 6
curr: 7
curr: 8
lag: 2
curr: 0
curr: 1
curr: 2
curr: 3
curr: 4
curr: 5
curr: 6
curr: 7
lag: 3
curr: 0
curr: 1
curr: 2
curr: 3
curr: 4
curr: 5
curr: 6
lag: 4
curr: 0
curr: 1
curr: 2
curr: 3
curr: 4
curr: 5
lag: 5
curr: 0
curr: 1
curr: 2
curr: 3
curr: 4
lag: 6
curr: 0
curr: 1
curr: 2
curr: 3
lag: 7
curr: 0
curr: 1
curr: 2
lag: 8
curr: 0
curr: 1
lag: 9
curr: 0
[      0.           56426.98120794  192868.75919739  374484.83623557
  583811.66540588  819319.38226769 1095007.68972109 1441040.89607656
 1905422.10632966 2556706.56664059]


In [10]:
test_class = ViscosityHelfand(test_u.atoms)
test_class.run()

Diff: [[[   -8.    -8.    -8.]]

 [[  -56.   -56.   -56.]]

 [[ -152.  -152.  -152.]]

 [[ -296.  -296.  -296.]]

 [[ -488.  -488.  -488.]]

 [[ -728.  -728.  -728.]]

 [[-1016. -1016. -1016.]]

 [[-1352. -1352. -1352.]]

 [[-1736. -1736. -1736.]]]
Sq_diff: [[1.920000e+02]
 [9.408000e+03]
 [6.931200e+04]
 [2.628480e+05]
 [7.144320e+05]
 [1.589952e+06]
 [3.096768e+06]
 [5.483712e+06]
 [9.041088e+06]]
Diff: [[[  -64.   -64.   -64.]]

 [[ -208.  -208.  -208.]]

 [[ -448.  -448.  -448.]]

 [[ -784.  -784.  -784.]]

 [[-1216. -1216. -1216.]]

 [[-1744. -1744. -1744.]]

 [[-2368. -2368. -2368.]]

 [[-3088. -3088. -3088.]]]
Sq_diff: [[1.2288000e+04]
 [1.2979200e+05]
 [6.0211200e+05]
 [1.8439680e+06]
 [4.4359680e+06]
 [9.1246080e+06]
 [1.6822272e+07]
 [2.8607232e+07]]
Diff: [[[ -216.  -216.  -216.]]

 [[ -504.  -504.  -504.]]

 [[ -936.  -936.  -936.]]

 [[-1512. -1512. -1512.]]

 [[-2232. -2232. -2232.]]

 [[-3096. -3096. -3096.]]

 [[-4104. -4104. -4104.]]]
Sq_diff: [[  139968.]
 [  762048.]

In [11]:
test_res == test_class.results.timeseries

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True])